In [1]:
from tensorflow.keras.models import load_model
from flask import Flask,request, jsonify
import os
import numpy as np
import uuid
app = Flask(__name__)

In [2]:
Expected = {
"cylinders":{"min":3,"max":8},
"displacement":{"min":68.0,"max":455.0},
"horsepower":{"min":46.0,"max":230.0},
"weight":{"min":1613,"max":5140},
"acceleration":{"min":8.0,"max":24.8},
"year":{"min":70,"max":82},
"origin":{"min":1,"max":3}
}

In [3]:
os.chdir(r'C:\Users\danie\Desktop\Flask UdemyKurs')

model = load_model(os.path.join(os.getcwd(),"mpg_model.h5"))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 25)                200       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                260       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 471
Trainable params: 471
Non-trainable params: 0
_________________________________________________________________


In [5]:
@app.route('/api', methods=['POST'])
def mpg_prediction():
    content = request.json
    errors = []
    for name in content:
        if name in Expected:
            expected_min = Expected[name]['min']
            expected_max = Expected[name]['max']
            value = content[name]
            if value < expected_min or value > expected_max:
                errors.append(f"Out of bounds: {name}, has value of: {value}, but it should be between {expected_min} and {expected_max}.")
        else:
            errors.append(f"Unexpected field: {name}.")
    for name in Expected:
        if name not in content:
            errors.append(f"Missing_value:{name}")
    if len(errors)<1:
        x= np.zeros((1,7))
        
        x[0,0] = content['cylinders']
        x[0,1] = content['displacement']
        x[0,2] = content['horsepower']
        x[0,3] = content['weight']
        x[0,4] = content['acceleration']
        x[0,5] = content['year']
        x[0,6] = content['origin']
        
        prediction = model.predict(x)
        mpg = float(prediction[0])
        response = {"id": str(uuid.uuid4()), "mpg": mpg,"errors":errors}
    else:
        response = {"id": str(uuid.uuid4()), "errrors": errors}
        
    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
                

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
